# Analyzing SMF 110 Monitoring Data

The following demo analyzes one day's worth of SMF 110 records. SMF Records are split into three main sections. SMF Header, SMF Product Section, and the Dictionary Data Section.  There are three fields that we are interested in: **USRCPUT**, **SMFMNRST**, and **SMFMNSPN**. 

* **USRCPUT** gives us the total CPU time for a user task and is contained within the Dictionary Data Section. Mainframe Data Studio (MDS), divides USRCPUT into individual components: USRCPUT_TIMER, USRCPUT_FLAG, and USRCPUT_COUNT. USRCPUT_TIMER is the component we care about because it captures the CPU time. MDS also converts this value from STCK format to decimal seconds.
* **SMFMNRST** and SMFMNRSD gives the job date and timestamp and are both found in the SMF Product Section. MDS converts the time field, SMFMNRST to a DB2 timestamp and thus, this field is sufficient enough to show both the date and time.
* **SMF_SID** The system identifier or the z/OS LPAR.
* **DB2REQCT** The total number of DB2® EXEC SQL and Instrumentation Facility Interface (IFI) requests issued by the user task.
* **WMQREQCT** The total number of WebSphere® MQ requests issued by the user task.
* **SMFMNSPN** identifies the CICS Region and is within the product section.

Using Spark and Python data analysis tools, we can capture insightful information on CPU consumption. 

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

#Remove font warnings
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore", category=PendingDeprecationWarning)

# Data Extraction

**Please Read** For extracting SMF110 data you have two options:

**1.** Via MDS. You can use SMF110 datasets on your z/os system. If doing it this way, please run the code cells underneath "SMF Data Extraction using Apache Spark and MDS."


**2.** Via JSON. You can use our sample SMF 110 data in our github titled "smf110.json" and run the cell blocks under "SMF Data Extraction using Pandas"

## 1) SMF Data Extraction using Apache Spark and MDS

In [ ]:
#Entry-point to Spark.
spark = SparkSession.builder.appName("SMF110-Demo").master("local[*]").getOrCreate()

In [ ]:
#Retrieving credentials to access SMF data

#Create a file with the password (first line of the file) needed to access the system where
#MDS is. Provide the absolute path in the field below.

CREDENTIALS_PATH = ""
def get_credentials():
    with open(CREDENTIALS_PATH) as f:
        password = f.readline()
    return password

password = get_credentials()

In [ ]:
#We use jdbc and DvDriver for accessing the DB2 database and retrieving the SMF data.

#The FQDN or IP address of the server where the database is for accessing SMF data.
ZOS_SYSTEM = ""

#Please provide the username in the "url" option value field 
#where it says "PROVIDE-USERNAME-HERE".
def get_smf_data(dbtable):
    DFReader = spark.read.format("jdbc") \
        .option("driver", "com.rs.jdbc.dv.DvDriver") \
        .option("url", "jdbc:rs:dv://" + ZOS_SYSTEM + ";DBTY=DVS;user=PROVIDE-USERNAME-HERE;password=" + password) \
        .option("dbtable", dbtable)
    smfData = DFReader.load()
    return smfData

#The database table names defined here are the virtual mappings we created for the datasets. 
#Notice that we need to grab fields from both the CICS dictionary and product section.

#The SMF110 dataset name. Please make sure to replace . with _ within the name.
DATASET_NAME = ""
cics_section = get_smf_data("SMF_1100P_PERFORMANCE__" + DATASET_NAME)
product_section = get_smf_data("SMF_1100P__" + DATASET_NAME)
cics_alias = cics_section.alias("cics_alias")
product_alias = product_section.alias("product_alias")
joined_df = cics_alias.join( \
    product_alias, col("cics_alias.PARENT_KEY") == col("product_alias.CHILD_KEY"), "inner")

#Tran and Trannum here are the Transaction Identification and 
#the Transaction Identification Number
filtered_df = joined_df.select( \
                 "SMFMNSPN", "TRAN", "TRANNUM", "SMF_SID","USRCPUT_TIMER", "USRCPUT_FLAG",\
                               "USRCPUT_COUNT","SMFMNRSD", "SMFMNRST","DB2REQCT", "WMQREQCT")

#Transform Spark dataframe to Pandas dataframe.
smf110_df = filtered_df.limit(80000).toPandas()

## 2) SMF Data Extraction using Pandas

In [ ]:
#Provide the path of the smf110.json location.
SMF110_PATH = ""
smf110_df = pd.read_json(SMF110_PATH)
smf110_df['SMFMNRST'] = pd.to_datetime(smf110_df['SMFMNRST'],unit='ms')

# Data Cleaning

In [ ]:
#Convert datatypes.
smf110_df['TRANNUM'] = smf110_df['TRANNUM'].astype(int)
smf110_df['USRCPUT_TIMER'] = smf110_df['USRCPUT_TIMER'].astype(float)
smf110_df['USRCPUT_COUNT'] = smf110_df['USRCPUT_COUNT'].astype(int)

orig_smf110_df = smf110_df.copy(deep=True)

#only keep the CICS regions i.e. SMFMNSPN starting with "CICS".
smf110_df = smf110_df[smf110_df.SMFMNSPN.str.contains("CICS") == True]

smf110_df.head(10)

# Exploratory Analysis

In [ ]:
print("The CICS Regions within dataset: ")
orig_smf110_df.SMFMNSPN.unique()

In [ ]:
print("The number of user tasks in this dataset: " + str(len(smf110_df)))
print('Total CPU Time in Seconds: {:2f}'.format(smf110_df['USRCPUT_TIMER'].sum()))
print('Total CPU Time in Hours: {:2f}'.format(smf110_df['USRCPUT_TIMER'].sum() / 3600))
print('Total DB2 Requests: {}'.format(smf110_df['DB2REQCT'].sum()))
print('Total WMQ Requests: {}'.format(smf110_df['WMQREQCT'].sum()))

In [ ]:
# Generate descriptive analysis for columns with float64 datatype. In our case, USRCPUT_TIMER.
smf110_df.describe(include=[np.float64])

## Determining Average Length of User Tasks' CPU Times

In [ ]:
%matplotlib inline
plt.rcParams["patch.force_edgecolor"] = True
plt.figure(figsize=(16,8))
plt.ylabel("Number of User Tasks", fontsize=15)
plt.xlabel("CPU Time in Seconds", fontsize=15)
plt.title("Determining Average Length of User Tasks' CPU Times", fontsize=15)
plt.hist(smf110_df['USRCPUT_TIMER'], bins=50, range=[0,.0001])
plt.show()

## Total Number of User Tasks Per CICS Region and System

In [ ]:
cics_run_per_region = smf110_df['SMFMNSPN'].value_counts()
plt.figure(figsize=(16,8))
cics_run_per_region.plot.barh(colormap='Paired')
plt.legend(smf110_df['SMF_SID'])
plt.xlabel("Number of User Tasks", fontsize=15)
plt.ylabel("CICS Region", fontsize=15)
plt.title("Total Number of User Tasks Per CICS Region, Per System", fontsize=15)
plt.show()

## Total CPU Time (s) Accumulated Per CICS Region and System

In [ ]:
cpu_time_per_region = smf110_df.groupby(['SMFMNSPN'])['USRCPUT_TIMER'].sum()

plt.figure(figsize=(16,8))
cpu_time_per_region.plot.barh(colormap='Paired')
plt.legend(smf110_df['SMF_SID'])
plt.xlabel("Total CPU time in Seconds", fontsize=15)
plt.ylabel("CICS Region", fontsize=15)
plt.title("Total CPU Time (s) Accumulated Per CICS Region, Per System", fontsize=15)
cpu_time_per_region

# Use Case

Lets zoom in on the 2 CICS Regions that accumulate the most CPU Time.

In [ ]:
smf110_df[smf110_df['SMFMNSPN'] == 'CICS3AAB']['TRAN'].value_counts().head(5)

In [ ]:
smf110_df[smf110_df['SMFMNSPN'] == 'CICS6AAA']['TRAN'].value_counts().head(5)

In [ ]:
#We take a few of the transaction identifiers within the two CICS Regions of interest.
smf110_filtered_tran_df = smf110_df[smf110_df['TRAN'].str.contains('^Y[1-5]95$|MAP')]
smf110_filtered_tran_df = smf110_filtered_tran_df [smf110_df['SMFMNSPN'].str.contains('CICS3AAB|CICS6AAA')]

In [ ]:
#The Transactions of interest.
smf110_filtered_tran_df.TRAN.unique()

In [ ]:
#The CICS Regions of interest.
smf110_filtered_tran_df.SMFMNSPN.unique()

## Total CPU Time (s) Accumulated Per Transaction, Per System

In [ ]:
cpu_time_per_trans = smf110_filtered_tran_df.groupby(['TRAN', 'SMF_SID'])['USRCPUT_TIMER'].sum().unstack()
cpu_time_per_trans.plot.barh(figsize=(16,8),colormap='Paired')
plt.xlabel("Total CPU time in Seconds", fontsize=15)
plt.ylabel("Transaction Type", fontsize=15)
plt.title("Total CPU Time (s) Accumulated Per Transaction, Per LPAR", fontsize=15)
cpu_time_per_trans

## CICS Transaction Rate Per CICS Region and LPAR

In [ ]:
smf110_filtered_tran_df['DATETIME_SECOND'] = smf110_filtered_tran_df['SMFMNRST'].apply(lambda x: x.second)
trans_rate_per_region = smf110_filtered_tran_df[['SMF_SID','SMFMNSPN','DATETIME_SECOND', 'TRAN']].pivot_table(index=['SMF_SID','SMFMNSPN','DATETIME_SECOND'], columns=['TRAN'], aggfunc=len)
trans_rate_per_region = trans_rate_per_region.fillna(0)
sys_id = trans_rate_per_region.index.levels[0]
cics_regions = trans_rate_per_region.index.get_level_values(1)
num_days_cics_regions = dict()
for j in range(len(sys_id)):
    for i in range(len(cics_regions)):
        num_days_cics_regions[(sys_id[j],cics_regions[i])] = num_days_cics_regions.get((sys_id[j],cics_regions[i]), 0) + 1
start_ind = 0
cics_trans_df = pd.DataFrame(index= list(trans_rate_per_region.columns), columns=list(num_days_cics_regions.keys()))
for cics_region_per_sys in num_days_cics_regions:
    end_ind = start_ind + num_days_cics_regions[cics_region_per_sys]
    cics_rate = trans_rate_per_region.iloc[start_ind:end_ind,:].apply(lambda x : np.mean(x))
    cics_trans_df[cics_region_per_sys] = cics_rate
    start_ind = end_ind
cics_trans_df.plot(kind='bar', figsize=(18,8), colormap='Paired')
plt.xlabel("Transactions", fontsize=15)
plt.ylabel("CICS Transaction Rate (# of Transactions Per Second)", fontsize=15)
plt.title("CICS Transaction Rate Per CICS Region and System", fontsize=15)
cics_trans_df

## CPU Percentage per CICS Region and LPAR

In [ ]:
cpu_percentage_per_region = smf110_filtered_tran_df[['SMF_SID','SMFMNSPN', 'USRCPUT_TIMER']].pivot_table(index=['SMF_SID','SMFMNSPN'], values=['USRCPUT_TIMER'], aggfunc=np.sum)
#Percentage is against all the other CICS Regions in this dataset.
cpu_percentage_per_region = cpu_percentage_per_region.apply(lambda x : x / float(orig_smf110_df['USRCPUT_TIMER'].sum()))
cpu_percentage_per_region.plot(kind='bar', figsize=(15,8), legend=False, colormap='Paired')
plt.xlabel("(LPAR, CICS Region)", fontsize=15)
plt.ylabel("Total CPU Percentage (Decimal)", fontsize=15)
plt.title("Total CPU Percentage Per CICS Region and System", fontsize=15)
cpu_percentage_per_region